# Synthèse de textures par réseau convolutif (filtres aléatoires)

## Mise en Place

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO
from urllib.request import urlopen
import os

from utils import prep_img, denormalize, to_pil, randomize_layer_, randomize_model_, register_model_hooks, unregister_model_hooks

In [ ]:
# fetch images
texture_imgnames = ["bois.png", "briques.png", "mur.png",
                    "tissu.png", "nuages.png", "pebbles.jpg", "wall1003.png"]
#TODO use urllib instead 
#import wget
for fname in texture_imgnames:
    os.system("wget -c https://www.idpoisson.fr/galerne/mva/" + fname)

In [ ]:
# device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is", device)

## ranVGG

### VGG

In [ ]:
#TODO description de VGG

Dans un premier temps, nous chargeons le modèle VGG pré-entraîné afin d'évaluer notre implémentation sur un réseau dont les poids sont déjà optimisés:

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

### Reconstruction d'images

In [ ]:
#TODO description théorique  de la content loss

In [ ]:
def content_loss(cnn: nn.Module, layer_output_: "list[torch.Tensor]", target_out: torch.Tensor, synthetic_image: "torch.Tensor", weighing_factor: float = None):
    # uses the layer_output variable to get the output of the target layer
    # assumes correctly batched input (ie bchw dimensions)
    # assumes target of shape chw
    # make sure to un/register hooks before/after content_loss, using relu output
    # suitable to use inside a closure (see https://pytorch.org/docs/stable/optim.html)

    # step 1: forward-propagate and get the layer output from the mutable
    cnn(synthetic_image)
    synth_out = layer_output_[0] # add batch dimension
    
    # step 2: compute the loss
    n_feature_maps, feature_height, feature_width = synth_out.shape
    loss = F.mse_loss(synth_out, target_out)

    if weighing_factor is not None:
        loss *= weighing_factor

    return loss


In [ ]:
#TODO description de la reconstruction d'images

In [ ]:
def reconstruct_image(cnn: nn.Module, target_image: torch.Tensor, target_layer_idx: int, n_steps: int = 10, synth_std: float = 0.5):
    # return reconstructed image and final loss

    # on initialise la pré-umage avec un bruit blanc:
    synthetic_image = torch.normal(
        0,
        synth_std,
        target_image.shape,
        device=target_image.device,
        requires_grad=True
    )

    layer_output_, handles = register_model_hooks(cnn, [target_layer_idx])

    # le réseau ne va pas être modifié, donc on peut calculer la cible une fois pour toutes:
    cnn(target_image)
    target_out = torch.clone(layer_output_[0].detach()) # on considère la cible comme une vérité terrain

    optimizer = optim.LBFGS([synthetic_image], max_iter=20)

    loss_history = []

    for iter in range(n_steps):

        def closure():
            # zero out the gradients, else they'll accumulate in synthetic_image.grad
            optimizer.zero_grad()
            loss = content_loss(cnn, layer_output_, target_out, synthetic_image)
            loss.backward()  # backpropagate the loss to the input
            loss_history.append(loss.item())
            return loss

        optimizer.step(closure)

    final_loss = content_loss(cnn, layer_output_, target_out, synthetic_image)

    unregister_model_hooks(handles)

    return synthetic_image, final_loss, loss_history


Testons la reconstruction d'image; He *et al.* comparent leurs résultats aux sorties des couches de pooling de VGG, et nous allons donc utiliser la couche pooling3:

In [ ]:
input_image_name = "briques.png"
img_size = 512
target_layer = 18  # pool3

target_image = prep_img(input_image_name, img_size).to(device)
synthetic_image, loss, loss_history = reconstruct_image(
    cnn,
    target_image,
    target_layer,
    n_steps=100
)

fig, _ = plt.subplots(1, 2, figsize=(15, 10))
fig.axes[0].imshow(to_pil(target_image))
fig.axes[0].set_title("Original")
fig.axes[1].imshow(to_pil(synthetic_image))
fig.axes[1].set_title("Reconstruction")
fig.tight_layout()
plt.show()


Nous arrivons bien à reconstruire l'image d'origine.

In [ ]:
# plot loss history
fig, _ = plt.subplots(1, 1, figsize=(15, 10))
fig.axes[0].plot(loss_history)
fig.axes[0].set_xlabel("L-BFGS iterations")
fig.axes[0].set_ylabel("Content loss")
fig.axes[0].set_xscale('log')
plt.show()

Au vu de la courbe ci-dessus, la fonction de perte semble converger pour VGG autour de 100 itérations de L-BGFS; nous avons donc choisi `n_steps = 10` par défaut, ce qui correspond à 200 itérations de L-BFGS (`optim.LBFGS` effectue par défaut 20 itérations par étape).

### Construction de ranVGG

In [ ]:
#TODO stuff below here is for later use

In [ ]:
def gramm(tnsr: torch.Tensor) -> torch.Tensor:
    """Computes Gram matrix for the input batch tensor.
    Args: tnsr (torch.Tensor): input tensor of the Size([B, C, H, W]).
    Returns:  G (torch.Tensor): output tensor of the Size([B, C, C]).
    """
    b, c, h, w = tnsr.size()
    F = tnsr.view(b, c, h * w)
    G = torch.bmm(F, F.transpose(1, 2))
    G.div_(h * w)
    return G


def gram_loss(input: torch.Tensor, gramm_target: torch.Tensor, weight: float = 1.0):
    """Computes the MSE loss between the Gram matrix of the input and the target
    Gram matrix. 
    """
    loss = weight * F.mse_loss(gramm(input), gramm_target)
    return loss


#TODO define texture_loss

In [ ]:
def build_ranvgg_(cnn, img, n_samples=1000):
    # remplacement des poids du modèle par un bruit blanc:
    randomize_model_(cnn)
    cnn.requires_grad_(False)  # on empêche le modèle d'apprendre
    layers_to_build = [
        idx for idx, layer in enumerate(cnn) if isinstance(layer, nn.Conv2d)
    ]
    # on enregistre la sortie des couches qu'on construit
    outputs, handles = register_model_hooks(cnn, layers_to_build)
    # TODO do stuff here
    
    unregister_model_hooks(handles)  # on enlève les handles du modèle
    return cnn


In [ ]:
# layers to use in the texture synthesis:
TARGET_LAYERS = [1, 4, 9, 18, 27]  # 1rst Conv2d (after ReLU) and MaxPool2d
LAYER_NAMES = ["conv1", "pool1", "pool2", "pool3", "pool4"]